In [1]:
import numpy as np
# !pip install pyMCFSimplex
from pyMCFSimplex import *
import random
from scipy.optimize import linprog


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### ESTRAZIONE DEI DATI 
La prima funzione serve per estrarre i costi quadratici che si trovano nella diagonale di Q,
la seconda funzione serve per estrarre le seguenti quantità 
 - u,b,q
 - numero nodi
 - numero archi 


In [3]:
def matrice_Q(nome_file):
    vettore = []
    with open(nome_file, 'r') as file:
        dimensione = int(file.readline()) # la prima riga è la dimensione del vettore
        valori = file.readline().split() # dalla seconda riga 
        vettore = [float(valore) for valore in valori]
        
        
        #error check 
        if len(vettore) != dimensione:
            raise ValueError("Il numero di valori nel file non corrisponde alla dimensione specificata.")
    
    dimensione = len(vettore)
    matrice = np.zeros((dimensione, dimensione)) #creo matrice
    np.fill_diagonal(matrice, vettore)  #riempio la diagonale
    return matrice, vettore
   

nome_file = "1000/netgen-1000-1-1-a-a-s.qfc"
Q_d = matrice_Q(nome_file)[1]
Q = matrice_Q(nome_file)[0]

In [4]:
def leggi_file_dimacs(file_path):
    numero_nodi = 0
    numero_archi = 0
    u = []
    b = []
    q = []
    from_=[]
    to_=[]
    edges = []

    with open(file_path, 'r') as file:
        for line in file:
            parts = line.split()
            if len(parts) > 0:
                if parts[0] == 'p':
                    # Leggi il numero di nodi e archi dal problema
                    numero_nodi = int(parts[2])
                    numero_archi = int(parts[3])
                    # Inizializza il vettore di supply con zeri
                    b = [0] * numero_nodi
                elif parts[0] == 'n':
                    # Leggi i valori di supply per i nodi
                    nodo_id = int(parts[1])
                    supply = int(parts[2])
                    # Assegna il valore di supply al nodo corrispondente
                    b[nodo_id - 1] = supply
                elif parts[0] == 'a':
                    # Leggi l'arc e il suo costo
                    from_node = int(parts[1])
                    to_node = int(parts[2])
                    max_capacity = int(parts[4])
                    costo = int(parts[5])  # Ora leggiamo il costo corretto
                    from_.append(from_node)
                    to_.append(to_node)
                    u.append(max_capacity)
                    q.append(costo)
                    edges.append((from_node , to_node ))

    return numero_nodi, numero_archi, u, b, q, edges,from_,to_

# Esempio di utilizzo
file_path = '1000/netgen-1000-1-1-a-a-s.dmx'  # Sostituisci con il percorso del tuo file DIMACS
numero_nodi, numero_archi, u, b, q, edges,from_ ,to_ = leggi_file_dimacs(file_path)

# creazione della matrice E 
E = np.zeros((numero_nodi, len(edges)), int)
#import pandas as pd
#E = pd.DataFrame(E)
#metto nomi delle colonne delle matrice E uguali agli archi
#E.columns = [edges]

### ALGORITMO 


In [5]:
#________________________
#      STEP 0 
#_______________________

#Generazione x_0 iniziali 
# def scale_vector(x, u):
    # scaled_x = []
    # for xi, ui in zip(x, u):
      #  scaling_factor = ui / max(abs(xi), 1e-6)
       # scaled_xi = xi * scaling_factor
        #scaled_x.append(scaled_xi)
    #return scaled_x


#x, residuals, _, _ = np.linalg.lstsq(E, b, rcond=None)
#scaled_x = scale_vector(x, u)

np.random.seed(42)
x_0 = []

for u_i in u:
    x_i = random.randint(0, u_i)  # 0 <= x_i <= u_i
    x_0.append(x_i)

In [6]:
#________________________
#      STEP 1 
#_______________________

# calcolo del gradiente 
gradient = 2 * np.dot(Q, x_0) + q
gradient = gradient.tolist() 
# calcolo delle x_bar 

#direzione 
# d = x_bar - x_0

In [ ]:
#________________________
#      STEP 2 
#_______________________

# determinazione alpha, step size 
# alpha= 2/(2+k)
#alpha= 2/(2+0)


In [ ]:
#________________________
#      STEP 3
#_______________________

# aggiornamento posizione 
#x_(k+1)=x_k + alpha_k *d_k


# check terminazione 

# k=k+1 

In [8]:
# lettura file esempio 
with open('1000/netgen-1000-1-1-a-a-s.dmx', 'r') as file:
    lines = file.readlines()

# Inizializza l'indice per il vettore gradient
gradient_index = 0

# Modifica le righe che iniziano con 'a'
for i in range(len(lines)):
    if lines[i].startswith('a'):
        # Dividi la riga in una lista di parole
        words = lines[i].split()
        # Sostituisci l'ultimo valore con il valore corrispondente nel vettore gradient
        words[-1] = str(gradient[gradient_index])
        # Incrementa l'indice del vettore gradient
        gradient_index += 1
        # Unisci le parole modificate per formare la nuova riga
        lines[i] = ' '.join(words)

# Scrivi il risultato nel nuovo file
with open('output3.dmx', 'w') as file:
    file.writelines(lines)

In [9]:
# MCFP problem transformed to integers and lists
nmx     = numero_nodi # max number of nodes
mmx     = numero_archi # max number of arcs
pn      = numero_nodi # current number of nodes
pm      = numero_archi# current number of arcs
pU      = u # column maxflow
pC      = gradient # column cost
pDfct   = b  # node deficit (supply/demand)
pSn     = from_ # column from
pEn     = to_ # column to

# call LoadNet() with the return values of the helper methods
# e.g. CreateDoubleArrayFromList(pU) takes a python list and returns a pointer to a 
# corresponding C array, that is passed as an argument to the method LoadNet()
mcf = MCFSimplex()
mcf.LoadNet(nmx, mmx, pn, pm, CreateDoubleArrayFromList(pU), CreateDoubleArrayFromList(pC),
            CreateDoubleArrayFromList(pDfct), CreateUIntArrayFromList(pSn),
            CreateUIntArrayFromList(pEn))

print( "Setting time..")
mcf.SetMCFTime()
mcf.SolveMCF()
if mcf.MCFGetStatus() == 0:
    print ("Optimal solution: %s" %mcf.MCFGetFO())
    print( "Time elapsed: %s sec " %(mcf.TimeMCF()))
else:
    print( "Problem unfeasible!")
    print( "Time elapsed: %s sec " %(mcf.TimeMCF()))


Setting time..
Problem unfeasible!
Time elapsed: 0.0 sec 


In [14]:
vettore_soluzione={}

In [15]:
def showModuleFunctionality():
   
    nmx = mcf.MCFnmax()
    mmx = mcf.MCFmmax()
    pn = mcf.MCFnmax()
    pm = mcf.MCFmmax()

    pU = []
    caps = new_darray(mmx)
    mcf.MCFUCaps(caps)
    for i in range(0, mmx):
        pU.append(darray_get(caps, i))

    pC = []
    costs = new_darray(mmx)
    mcf.MCFCosts(costs)
    for i in range(0, mmx):
        pC.append(darray_get(costs, i))

    pDfct = []
    supply = new_darray(nmx)
    mcf.MCFDfcts(supply)
    for i in range(0, nmx):
        pDfct.append(darray_get(supply, i))

    pSn = []
    pEn = []
    startNodes = new_uiarray(mmx)
    endNodes = new_uiarray(mmx)
    mcf.MCFArcs(startNodes, endNodes)
    for i in range(0, mmx):
        pSn.append(uiarray_get(startNodes, i) + 1)
        pEn.append(uiarray_get(endNodes, i) + 1)

    print("arc flow")
    length = mcf.MCFm()
    flow = new_darray(length)
    length = mcf.MCFn()
    nms = new_uiarray(length)
    mcf.MCFGetX(flow, nms)

   

    for i in range(0, length):
        print("flow", darray_get(flow, i), "arc", uiarray_get(nms, i))
        vettore_soluzione[uiarray_get(nms, i)] = darray_get(flow, i)

    print("node potentials")
    length = mcf.MCFn()
    costs = new_darray(length)
    mcf.MCFGetPi(costs, nms)
    for i in range(0, length):
        print("flow", darray_get(costs, i), "node", i + 1)

    print("reading graph - arcs")
    length = mcf.MCFm()
    startNodes = new_uiarray(length)
    endNodes = new_uiarray(length)
    mcf.MCFArcs(startNodes, endNodes)
    for i in range(0, length):
        print("Arc %s: start %s end %s" % (i, uiarray_get(startNodes, i) + 1, uiarray_get(endNodes, i) + 1))

    print("reading graph - costs")
    length = mcf.MCFm()
    costs = new_darray(length)
    mcf.MCFCosts(costs)
    for i in range(0, length):
        print("Arc %s: cost %s" % (i, darray_get(costs, i)))

    print("reading graph - capacities")
    length = mcf.MCFm()
    caps = new_darray(length)
    mcf.MCFUCaps(caps)
    for i in range(0, length):
        print("Arc %s: capacities %s" % (i, darray_get(caps, i)))

    print("reading nodes - supply/demand")
    length = mcf.MCFn()
    supply = new_darray(length)
    mcf.MCFDfcts(supply)
    for i in range(0, length):
        print("Node %s: demand %s" % (i + 1, darray_get(supply, i)))

    return vettore_soluzione  # Restituisci il vettore_soluzione alla fine della funzione


In [16]:
#from pyMCFSimplex import *
#print( "pyMCFSimplex Version '%s' successfully imported." % version())
#mcf = MCFSimplex()
#print ("MCFSimplex Class successfully instantiated.")
FILENAME = 'output3.dmx'
#print ("Loading network from DIMACS file %s.." % FILENAME)
f = open(FILENAME,'r')
inputStr = f.read()
f.close()
mcf.LoadDMX(inputStr)
print ("Setting time..")
mcf.SetMCFTime()
print ("Solving problem..")
mcf.SolveMCF()
if mcf.MCFGetStatus() == 0:
    print ("Optimal solution: %s" %mcf.MCFGetFO())
   
    print ("Time elapsed: %s sec " %(mcf.TimeMCF()))
else:
    print( "Problem unfeasible!")
    print ("Time elapsed: %s sec " %(mcf.TimeMCF()))

Setting time..
Solving problem..
Optimal solution: 556985.0
Time elapsed: 0.001 sec 


In [17]:
%%capture
showModuleFunctionality()

In [18]:
vettore_soluzione
sol_x=[0]*1000
for key in vettore_soluzione:
    if key<=1000:
    #print(key)
        sol_x[key-1]=vettore_soluzione[key]
print(sol_x)

[0, 0, 0, 0, 0, 0, 0, 0.0, 0, 0, 0.0, 0, 0, 0, 0.0, 0, 0, 6.0, 0.0, 0, 0, 11.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 18.0, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 21.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 37.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 35.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 29.0, 0, 0, 0, 0, 0, 0, 0, 21.0, 0, 0, 0, 0, 0, 0, 0, 0, 24.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0, 1